In [ ]:
import itertools
import ipywidgets as widgets

import numpy as np
from ipycanvas import Canvas, MultiCanvas

import bokeh
from bokeh.io import output_notebook, show, push_notebook

output_notebook(bokeh.resources.INLINE)

from bokeh.models import ColumnDataSource, Circle, Line
import bokeh.plotting

Some parameters that can be tweaked (carefully, though)

In [ ]:
width, height = 800, 600 # in pixels
n_sensors = 4

# horizontal_padding = 50 # in pixels
horizontal_padding = 100 # in pixels
vertical_padding = horizontal_padding

# the smaller, the easier the sensors get activated
obs_to_angle_sensitivity = 0.25

# tx_power = 1e3
tx_power = 2e3

sensors_fill_color = 'red'
position_fill_color = 'green'

Related to the model

In [ ]:
def obs(x_1: int, y_1: int, x_2: int, y_2) -> float:
    """
        Computes an observation given the x and y coordinates for two different positions.
    
    """
    
    return tx_power / np.sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2)

Functions to go from the model to the *canvas*

In [ ]:
def obs_to_angle(obs: float) -> float:
    """
        Turns a measure into an angle in radians.
    
    """
    
    return ((1. / (1. + np.exp(-obs_to_angle_sensitivity*obs))) - 0.5) / (1 - 0.5) * 2*np.pi

# Preliminary computations

In [ ]:
# number of sensors per axis
n_sensors_per_axis = int(np.sqrt(n_sensors))

# all the x coordinates of the sensors...
sensors_x_coordinates = np.linspace(horizontal_padding, width-horizontal_padding, n_sensors_per_axis)

# ...and all the y's
sensors_y_coordinates = np.linspace(vertical_padding, height-vertical_padding, n_sensors_per_axis)

# a list of tuples with the resulting positions
sensors_coordinates = list(itertools.product(sensors_x_coordinates, sensors_y_coordinates))

n_sensors = len(sensors_coordinates)

*Global* variables are used to keep track of the clicked positions.

In [ ]:
target_position = []
observations = []

# Canvas setup

In [ ]:
# canvas
canvas = MultiCanvas(n_canvases=2, width=800, height=600)

# default style settings
canvas[0].fill_style = sensors_fill_color
canvas[0].stroke_style = 'blue'

Handler for when mouse is *un-clicked*

In [ ]:
def handle_mouse_up(x, y):
    
    target_position.append((x,y))
    
    # a squared dot representing the clicked position
    canvas[1].fill_style = position_fill_color
    canvas[1].fill_rect(x, y, 10, 10)
    
#     print(sensors_coordinates)
    
    measurements = [None] * n_sensors
    
    for i_sensor, (x_sensor, y_sensor) in enumerate(sensors_coordinates):
        
        # the previous filling is "erased
        canvas[1].fill_style = 'white'
        canvas[1].fill_arc(x_sensor, y_sensor, 19, 0, 2*np.pi)
        
        measurements[i_sensor] = obs(x, y, x_sensor, y_sensor)
        
        # new filling
        canvas[1].fill_style = sensors_fill_color
        canvas[1].fill_arc(x_sensor, y_sensor, 19, 0, obs_to_angle(measurements[i_sensor]))
    
    # the new list with measurements from all the sensors is appended to previous observations
    observations.append(measurements)
    
    # for the sake of convenience
    observations_np = np.array(observations)
    
    x = np.arange(observations_np.shape[0])

    # for every plot source and handler, along with the newly added observations
    for source, handler, y in zip(sources, handlers, observations_np.T):
        
        source.data = {'x':x, 'y':y}
        push_notebook(handle=handler)

canvas[1].on_mouse_up(handle_mouse_up)

Sensors are plotted

In [ ]:
output_plots = []
figures, sources = [], []

# for every sensor's x-y coordinates
for x, y in sensors_coordinates:
    
    # the sensor is plotted (outlined)
    canvas[0].stroke_arc(x, y, 20, 0, 2*np.pi)
    
    # a new output widget is created for the corresponding plot
    output_plots.append(widgets.Output())
    
    figures.append(bokeh.plotting.figure(plot_width=400, plot_height=300, y_range=(0, 40)))
    sources.append(ColumnDataSource(data=dict(x=[0], y=[0])))
#     circle = Circle(x="x", y="y", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
#     figures[-1].add_glyph(sources[-1], circle)
    line = Line(x="x", y="y")
    figures[-1].add_glyph(sources[-1], line)

In [ ]:
columns = []

for i in range(0, n_sensors,n_sensors_per_axis):
    
    columns.append(widgets.VBox(output_plots[i:i+n_sensors_per_axis]))

grid = widgets.HBox(columns)

In [ ]:
display(grid)

handlers = []

for figure, output_plot in zip(figures, output_plots):
    
    with output_plot:
        
        handlers.append(show(figure, notebook_handle=True))

Handlers

In [ ]:
canvas